In [1]:
#Imports
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)

## CRISP-DM
![CRISP-DM](../reports/figures/CRISP_DM.png)

## Data Preparation
Relational Dataset

In [3]:
# Path to relational dataset
pth_rds= "../data/processed/COVID_relational_full.csv"
# Read data into dataframe
pd_rel= pd.read_csv(pth_rds, sep=";", parse_dates=[0])
# Sirt by dates
pd_rel= pd_rel.sort_values('date', ascending=True)

In [4]:
pd_rel.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
169,2020-01-22,no,"Korea, South",1.0
170,2020-01-22,no,Kosovo,0.0
171,2020-01-22,no,Kuwait,0.0
172,2020-01-22,no,Kyrgyzstan,0.0


In [5]:
pd_rel= pd_rel.reset_index(drop=True)
pd_rel= pd_rel.reset_index()

In [6]:
pd_rel.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [7]:
# Cut out test data
pd_rel_small= pd_rel[
    (
        (pd_rel['country'] == 'Nigeria') |
        (pd_rel['country'] == 'Germany')
    )&
    (
        pd_rel['date'] > '2020-08-01'
    )
]

In [8]:
pd_rel_small.tail()

,index,date,state,country,confirmed
54101,54101,2020-08-12,no,Nigeria,47743.0
54307,54307,2020-08-13,no,Germany,222281.0
54368,54368,2020-08-13,no,Nigeria,48116.0
54572,54572,2020-08-14,no,Germany,223791.0
54634,54634,2020-08-14,no,Nigeria,48445.0


In [9]:
# Group entries by country and search for the maximum
pd_rel_small.groupby(["country"]).agg(np.max)

,index,date,state,confirmed
country,,,,
Germany,54572,2020-08-14,no,223791.0
Nigeria,54634,2020-08-14,no,48445.0


In [10]:
# %load ../src/features/build_features.py
import numpy as np
from sklearn import linear_model
from scipy import signal

# Create Linear Regression Model
reg= linear_model.LinearRegression(fit_intercept= True)  


def get_doubling_rate_via_regression(in_array):
    """ Approximate the doubling time using linear regression.

    3 datapoints are used to approximate the number of days 
    it takes for the number of infected people to double at each point.

    Parameters:
    ----------
    in_array: List/ numpy Array
        input data

    Returns:
    -------
    doubling_time: double
    """
    
    # Assert output vector is 3 datapoints long
    assert len(in_array)==3
 
    y= np.array(in_array)
    # Calculate slope using central difference
    X= np.arange(-1,2).reshape(-1,1)

    # Fit data
    reg.fit(X,y)
    intercept= reg.intercept_
    slope= reg.coef_

    return intercept/slope


def rolling_regression(df_input, col="confirmed"):
    """ Roll over entries to approximate the doubling time using linear regression.

    Parameters:
    ----------
    df_input: pandas DataFrame
        input data
    col: string
        key to column which holds data entries

    Returns:
    -------
    result: pandas Series
    """
    
    days_back= 3
    
    result= df_input[col].rolling(
            window=days_back,
            min_periods=days_back
        ).apply(get_doubling_rate_via_regression, raw=False)
    
    return result


def savgol_filter(df_input, col='confirmed', window=5):
    """ Filter data using savgol filter.

    Parameters:
    ----------
    df_input: pandas DataFrame
        input data
    col: string
        key to column which holds data entries

    Returns:
    -------
    df_result: pandas DataFrame
        df_input with additional column with name col+"_filtered"
    """

    window=5
    degree=1

    df_result=df_input

    filter_in= df_input[col].fillna(0)
    result= signal.savgol_filter(
            np.array(filter_in), window, degree
        )

    df_result[col+ "_filtered"]= result
    return df_result
    




if __name__ == "__main__":
    # Test data
    test_data= np.array([2,4,6])
    # Expected result= 2
    result= get_doubling_rate_via_regression(test_data)
    assert(int(result[0]) == 2)

In [11]:
pd_rel_small.groupby(['state', 'country']).agg(np.max)

index       date  confirmed
state country                             
no    Germany  54572 2020-08-14   223791.0
      Nigeria  54634 2020-08-14    48445.0

#### Apply get_doubling_rate_via_regression() on country groups

In [12]:
pd_rel_small_DR= pd_rel_small.groupby(['state', 'country']).apply(rolling_regression).reset_index()

In [13]:
pd_rel_small_DR.tail()

,state,country,level_2,confirmed
21,no,Nigeria,53570,128.000000
22,no,Nigeria,53834,131.588593
23,no,Nigeria,54101,107.990868
24,no,Nigeria,54368,115.535916
25,no,Nigeria,54634,137.040836


In [14]:
pd_rel_small_DR= pd_rel_small_DR.rename(columns={'confirmed':'doubling_rate', 'level_2': 'index'})

In [15]:
pd_rel_small_result= pd.merge(pd_rel_small, pd_rel_small_DR[['index', 'doubling_rate']], on=['index'], how='left')

In [16]:
pd_rel_small_result.tail()

,index,date,state,country,confirmed,doubling_rate
21,54101,2020-08-12,no,Nigeria,47743.0,107.990868
22,54307,2020-08-13,no,Germany,222281.0,161.177186
23,54368,2020-08-13,no,Nigeria,48116.0,115.535916
24,54572,2020-08-14,no,Germany,223791.0,151.644156
25,54634,2020-08-14,no,Nigeria,48445.0,137.040836


#### Filtering data while grouping by state and country

In [17]:
pd_rel_small_filtered= pd_rel_small[['state', 'country', 'confirmed']].groupby(['state', 'country']).apply(savgol_filter).reset_index()

In [18]:
pd_rel_small_filtered[['index', 'confirmed_filtered']].tail()

,index,confirmed_filtered
21,54101,47692.2
22,54307,222326.5
23,54368,48090.4
24,54572,223657.2
25,54634,48488.6


In [19]:
pd_rel_small_result= pd.merge(pd_rel_small_result, pd_rel_small_filtered[['index', 'confirmed_filtered']], on=['index'], how='left')

In [20]:
pd_rel_small_result.tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
21,54101,2020-08-12,no,Nigeria,47743.0,107.990868,47692.2
22,54307,2020-08-13,no,Germany,222281.0,161.177186,222326.5
23,54368,2020-08-13,no,Nigeria,48116.0,115.535916,48090.4
24,54572,2020-08-14,no,Germany,223791.0,151.644156,223657.2
25,54634,2020-08-14,no,Nigeria,48445.0,137.040836,48488.6


## Doubling rate on Full dataset

In [21]:
pd_DR_rel= pd_rel[['state', 'country', 'confirmed']].groupby(['state', 'country']).apply(rolling_regression).reset_index()

In [22]:
pd_DR_rel.tail()

,state,country,level_2,confirmed
54791,no,Zimbabwe,53598,53.842004
54792,no,Zimbabwe,53796,56.074951
54793,no,Zimbabwe,54093,66.478161
54794,no,Zimbabwe,54328,56.980620
54795,no,Zimbabwe,54795,55.698324


In [23]:
pd_DR_rel= pd_DR_rel.rename(columns={'confirmed': 'doubling_rate', 'level_2': 'index'})

In [24]:
pd_DR_rel.tail()

,state,country,index,doubling_rate
54791,no,Zimbabwe,53598,53.842004
54792,no,Zimbabwe,53796,56.074951
54793,no,Zimbabwe,54093,66.478161
54794,no,Zimbabwe,54328,56.980620
54795,no,Zimbabwe,54795,55.698324


In [25]:
pd_rel.tail()

,index,date,state,country,confirmed
54791,54791,2020-08-14,no,Barbados,148.0
54792,54792,2020-08-14,no,Belarus,69308.0
54793,54793,2020-08-14,no,Belgium,77113.0
54794,54794,2020-08-14,no,Albania,7117.0
54795,54795,2020-08-14,no,Zimbabwe,5072.0


### Merge Big relational database with doubling rate results

In [26]:
pd_rel_result= pd.merge(pd_rel, pd_DR_rel[['index', 'doubling_rate']], on=['index'], how='left')

In [27]:
pd_rel_result[pd_rel_result['country']== 'Nigeria'].head()

,index,date,state,country,confirmed,doubling_rate
102,102,2020-01-22,no,Nigeria,0.0,NaN
367,367,2020-01-23,no,Nigeria,0.0,NaN
636,636,2020-01-24,no,Nigeria,0.0,NaN
901,901,2020-01-25,no,Nigeria,0.0,NaN
1165,1165,2020-01-26,no,Nigeria,0.0,NaN
1433,1433,2020-01-27,no,Nigeria,0.0,NaN
1699,1699,2020-01-28,no,Nigeria,0.0,NaN
1963,1963,2020-01-29,no,Nigeria,0.0,NaN
2231,2231,2020-01-30,no,Nigeria,0.0,NaN
2498,2498,2020-01-31,no,Nigeria,0.0,NaN


## Filtering on Big Relational Dataset

In [28]:
pd_rel_result.tail()

,index,date,state,country,confirmed,doubling_rate
54791,54791,2020-08-14,no,Barbados,148.0,72.666667
54792,54792,2020-08-14,no,Belarus,69308.0,671.886731
54793,54793,2020-08-14,no,Belgium,77113.0,104.115962
54794,54794,2020-08-14,no,Albania,7117.0,46.455556
54795,54795,2020-08-14,no,Zimbabwe,5072.0,55.698324


In [29]:
# Confirmed filtered
pd_rel_filtered= pd_rel[['state', 'country', 'confirmed']].groupby(['state','country']).apply(savgol_filter)

In [30]:
pd_rel_filtered= pd_rel_filtered.reset_index()

In [31]:
pd_rel_filtered.tail()

,index,state,country,confirmed,confirmed_filtered
54791,54791,no,Barbados,148.0,146.8
54792,54792,no,Belarus,69308.0,69297.0
54793,54793,no,Belgium,77113.0,76949.6
54794,54794,no,Albania,7117.0,7114.8
54795,54795,no,Zimbabwe,5072.0,5068.2


In [32]:
pd_rel_result= pd.merge(pd_rel_result, pd_rel_filtered[['index', 'confirmed_filtered']], on=['index'], how='left')

In [33]:
pd_rel_result.tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
54791,54791,2020-08-14,no,Barbados,148.0,72.666667,146.8
54792,54792,2020-08-14,no,Belarus,69308.0,671.886731,69297.0
54793,54793,2020-08-14,no,Belgium,77113.0,104.115962,76949.6
54794,54794,2020-08-14,no,Albania,7117.0,46.455556,7114.8
54795,54795,2020-08-14,no,Zimbabwe,5072.0,55.698324,5068.2


### Filtered Doubling Rate

In [34]:
pd_DR_filtered= pd_rel_result.groupby(['state', 'country']).apply(rolling_regression, col='confirmed_filtered')

In [35]:
pd_DR_filtered= pd_DR_filtered.reset_index()

In [36]:
pd_DR_filtered= pd_DR_filtered.rename(columns={'level_2': 'index', 'confirmed_filtered': 'doubling_rate_filtered'})

In [37]:
pd_DR_filtered.tail()

,state,country,index,doubling_rate_filtered
54791,no,Zimbabwe,53598,50.442273
54792,no,Zimbabwe,53796,55.248541
54793,no,Zimbabwe,54093,57.519491
54794,no,Zimbabwe,54328,58.863545
54795,no,Zimbabwe,54795,60.807317


In [38]:
# Merge results
pd_rel_result= pd.merge(pd_rel_result, pd_DR_filtered[['index', 'doubling_rate_filtered']], on=['index'], how='left')

In [39]:
pd_rel_result.tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered
54791,54791,2020-08-14,no,Barbados,148.0,72.666667,146.8,111.923077
54792,54792,2020-08-14,no,Belarus,69308.0,671.886731,69297.0,752.228261
54793,54793,2020-08-14,no,Belgium,77113.0,104.115962,76949.6,123.735938
54794,54794,2020-08-14,no,Albania,7117.0,46.455556,7114.8,47.831846
54795,54795,2020-08-14,no,Zimbabwe,5072.0,55.698324,5068.2,60.807317


#### Cleanup doubling rates

In [40]:
DR_mask= pd_rel_result['confirmed']>100

In [41]:
DR_mask

0        False
1        False
2        False
3        False
4        False
         ...  
54791     True
54792     True
54793     True
54794     True
54795     True
Name: confirmed, Length: 54796, dtype: bool

In [42]:
pd_rel_result['doubling_rate']= pd_rel_result['doubling_rate'].where(DR_mask, other=np.NaN)

In [43]:
pd_rel_result['doubling_rate_filtered']= pd_rel_result['doubling_rate_filtered'].where(DR_mask, other=np.NaN)

In [44]:
pd_rel_result.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0,NaN


In [45]:
# Save
pd_rel_result.to_csv('../data/processed/COVID_final_set.csv', sep=';', index=False)